In [1]:
%load_ext autoreload
%autoreload 2
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import pandas as pd

import drugz as dz

You can run functions individually

In [4]:
# import test dataset

# func load_reads(filepath, index_column, genes_to_remove)
data = dz.load_reads(
    "./sgRNA_count.txt", index_column=0, genes_to_remove=["LacZ, luciferase, EGFR"]
)
data.head()


,Gene,T0,T15_A_control,T15_B_control,T15_C_control,T15_A_olaparib,T15_B_olaparib,T15_C_olaparib
sgRNA,,,,,,,,
A1BG_CACCTTCGAGCTGCTGCGCG,A1BG,313,235,47,337,428,115,340
A1BG_AAGAGCGCCTCGGTCCCAGC,A1BG,99,8,1,13,26,5,28
A1BG_TGGACTTCCAGCTACGGCGC,A1BG,650,336,74,185,392,193,304
A1BG_CACTGGCGCCATCGAGAGCC,A1BG,718,192,34,296,178,69,185
A1BG_GCTCGGGCTTGTCCACAGGA,A1BG,180,230,29,122,394,148,364


In [5]:
# normalize readcounts

# func normalize_readconts(reads, control_samples, treatment_samples)
norm_data = dz.normalize_readcounts(
    data,
    ["T15_A_control", "T15_B_control", "T15_C_control"],
    ["T15_A_olaparib", "T15_B_olaparib", "T15_C_olaparib"],
)
norm_data.head()


,T15_A_olaparib,T15_B_olaparib,T15_C_olaparib,T15_A_control,T15_B_control,T15_C_control
sgRNA,,,,,,
A1BG_CACCTTCGAGCTGCTGCGCG,1.610233e+06,7.855191e+05,1.377076e+06,1.172070e+06,9.437751e+05,1.787798e+06
A1BG_AAGAGCGCCTCGGTCCCAGC,9.781791e+04,3.415301e+04,1.134062e+05,3.990025e+04,2.008032e+04,6.896552e+04
A1BG_TGGACTTCCAGCTACGGCGC,1.474793e+06,1.318306e+06,1.231268e+06,1.675810e+06,1.485944e+06,9.814324e+05
A1BG_CACTGGCGCCATCGAGAGCC,6.696764e+05,4.713115e+05,7.492912e+05,9.576060e+05,6.827309e+05,1.570292e+06
A1BG_GCTCGGGCTTGTCCACAGGA,1.482318e+06,1.010929e+06,1.474281e+06,1.147132e+06,5.823293e+05,6.472149e+05


In [6]:
# calculate fold change

# func calculate_fold_change(reads, normalized_counts, control_samples, treatment_samples, pseudocount, replicate):

lfc = []
# for each replicate
for i in range(2):
    lfc.append(
        dz.calculate_fold_change(
            data,
            norm_data,
            ["T15_A_control", "T15_B_control", "T15_C_control"],
            ["T15_A_olaparib", "T15_B_olaparib", "T15_C_olaparib"],
            5,
            i,
        )
    )
lfc[0]

,GENE,T15_A_control,eb_std_0,zscore_fc_0,fc_0
A1CF_CGTGGCTATTTGGCATACAC,A1CF,452,0.0,0.0,-0.034625
A1BG_TGGACTTCCAGCTACGGCGC,A1BG,336,0.0,0.0,-0.184346
A1BG_CAAGAGAAAGACCACGAGCA,A1BG,300,0.0,0.0,-0.119857
A1BG_CACCTTCGAGCTGCTGCGCG,A1BG,235,0.0,0.0,0.458209
A1BG_GCTCGGGCTTGTCCACAGGA,A1BG,230,0.0,0.0,0.369821
A1BG_CACTGGCGCCATCGAGAGCC,A1BG,192,0.0,0.0,-0.515965
A1CF_GACATGGTATTGCAGTAGAC,A1CF,183,0.0,0.0,-0.329980
A1CF_GGTATACTCTCCTTGCAGCA,A1CF,69,0.0,0.0,0.069698
A1BG_AAGAGCGCCTCGGTCCCAGC,A1BG,8,0.0,0.0,1.293594


In [7]:
# calculate the smoothed variance

# func empirical_bayes(fold_change, half_window_size, no_of_guides, fc_replicate_id,
# empirical_bayes_id, fc_zscore_id)
for i in range(len(lfc)):
    lfc[i] = dz.empirical_bayes(
        lfc[i], 5, 9, "fc_" + str(i), "eb_std_" + str(i), "zscore_fc_" + str(i)
    )
lfc[0]


(                           GENE  T15_A_control  eb_std_0  zscore_fc_0  \
 A1CF_CGTGGCTATTTGGCATACAC  A1CF            452  0.540492    -0.064062   
 A1BG_TGGACTTCCAGCTACGGCGC  A1BG            336  0.540492    -0.341071   
 A1BG_CAAGAGAAAGACCACGAGCA  A1BG            300  0.540492    -0.221756   
 A1BG_CACCTTCGAGCTGCTGCGCG  A1BG            235  0.540492     0.847764   
 A1BG_GCTCGGGCTTGTCCACAGGA  A1BG            230  0.540492     0.684232   
 A1BG_CACTGGCGCCATCGAGAGCC  A1BG            192  0.540492    -0.954621   
 A1CF_GACATGGTATTGCAGTAGAC  A1CF            183  0.540492    -0.610518   
 A1CF_GGTATACTCTCCTTGCAGCA  A1CF             69  0.540492     0.128953   
 A1BG_AAGAGCGCCTCGGTCCCAGC  A1BG              8  0.540492     2.393365   
 
                                fc_0  
 A1CF_CGTGGCTATTTGGCATACAC -0.034625  
 A1BG_TGGACTTCCAGCTACGGCGC -0.184346  
 A1BG_CAAGAGAAAGACCACGAGCA -0.119857  
 A1BG_CACCTTCGAGCTGCTGCGCG  0.458209  
 A1BG_GCTCGGGCTTGTCCACAGGA  0.369821  
 A1BG_CACTGGCGCCATCGAGAG


or, you can pass the arguments and run the whole analysis at once

In [8]:
# define the Arguements class (more convinient since iPython doesn't recognize argparse arguments)
# these are user-specified arguments

# infile = input readcounts matrix
# drugz_out_file = name of a file in which you will write the drugz results
# control_samples = the names of control samples (included in column names)
# drug_samples = the names of drug-treated samples (included in column names)
# unpaired = unpaired approach - compares mean(treated samples) to mean(control samples)
# pseudocount = counts added to the observed readscounts, default = 5
# half_window_size = size of the first bin and half the size of the inital sample (window) to estimate std,
#                    default = 500 (for whole genome screens)


class Args:
    infile = "./sgRNA_count.txt"
    drugz_output_file = "./drugz_results.txt"
    fc_outfile = "./fc_results.txt"
    control_samples = "T15_A_control,T15_B_control,T15_C_control"
    drug_samples = "T15_A_olaparib,T15_B_olaparib,T15_C_olaparib"
    remove_genes = "LacZ,luciferase,EGFR"
    unpaired = False
    pseudocount = 5
    half_window_size = 5  # 5 because of the size of test data set (sgRNA_count.txt = 9 guides (i.e. rows))

In [10]:
drugz_results = dz.drugZ_analysis(Args())


INFO:drugz:Initiating analysis
INFO:drugz:Loading the read count matrix
INFO:drugz:Normalizing read counts
INFO:drugz:Calculating raw fold change for replicate 1
INFO:drugz:Caculating smoothed Empirical Bayes estimates of stdev for replicate 1
INFO:drugz:Caculating guide-level Zscores for replicate 1
INFO:drugz:Calculating raw fold change for replicate 2
INFO:drugz:Caculating smoothed Empirical Bayes estimates of stdev for replicate 2
INFO:drugz:Caculating guide-level Zscores for replicate 2
INFO:drugz:Calculating raw fold change for replicate 3
INFO:drugz:Caculating smoothed Empirical Bayes estimates of stdev for replicate 3
INFO:drugz:Caculating guide-level Zscores for replicate 3
INFO:drugz:Caculating gene-level Zscores
INFO:drugz:Writing output file paired results


In [11]:
drugz_results


,sumZ,numObs,normZ,pval_synth,rank_synth,fdr_synth,pval_supp,rank_supp,fdr_supp
GENE,,,,,,,,,
A1CF,1.640558,9.0,-1.0,0.158655,1,0.317311,0.841345,2,0.841345
A1BG,3.095789,18.0,1.0,0.841345,2,0.841345,0.158655,1,0.317311



Option 2. Run the analysis as a program and specify the output file (e.g. drugZ_test.txt)¶


In [ ]:
%run drugz.py -i sgRNA_count.txt -o drugZ_results.txt -c T15_A_control,T15_B_control,T15_C_control -x T15_A_olaparib,T15_B_olaparib,T15_C_olaparib --half_window_size 5